In [1]:
import tensorflow as tf
# Basic RNNs in Tensorflow, at each time, every neuron receives both 
# the input vector and the previous output vector
n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32,[None,n_inputs])
X1 = tf.placeholder(tf.float32,[None,n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs,n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1,n_neurons],dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0,Wx)+b)
Y1 = tf.tanh(tf.matmul(Y0,Wy)+tf.matmul(X1,Wx)+b)

In [2]:
import numpy as np
init = tf.global_variables_initializer()
# Mini-batch
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) # t = 0
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]]) # t = 1

with tf.Session() as sess:
    sess.run(init)
    Y0_val, Y1_val = sess.run([Y0,Y1],feed_dict={X0: X0_batch,X1: X1_batch})
    
    print(Y0_val)
    print(Y1_val)

[[-0.98983276  0.95793128 -0.43150836  0.61514908 -0.77770168]
 [-1.          0.99990988 -0.8087526   0.94929969 -1.        ]
 [-1.          0.99999982 -0.94523847  0.99433792 -1.        ]
 [-0.87197351 -0.99839008  0.99999183  0.99922323 -1.        ]]
[[-1.          0.99999785 -0.96192461  0.2513243  -1.        ]
 [ 0.99727625 -0.77777004 -0.75489521 -0.98914921  0.63602072]
 [-0.9999994   0.99597675 -0.95873678 -0.80752027 -1.        ]
 [-0.98241305  0.96511012 -0.98859769 -0.63796723 -1.        ]]


In [7]:
# Static Unrolling Through Time with sttatic_rnn().The following code creates exact the same model
# like above, but is more simpler to write, especially when time steps are big.
#X0 = tf.placeholder(tf.float32,[None,n_inputs])
#X1 = tf.placeholder(tf.float32,[None,n_inputs])
n_steps = 2
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
X_seqs = tf.unstack(tf.transpose(X,perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
# there are several cell exists, so made its a name to distinguish
with tf.variable_scope('B1'):
    output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell,X_seqs,dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs),perm=[1,0,2])

X_batch = np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
])


In [9]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    print(outputs_val)

[[[-0.71581197 -0.55254775  0.86902887  0.65668207  0.9197892 ]
  [-1.         -0.99981415  1.         -0.12912703  0.99998623]]

 [[-0.99986368 -0.98185688  0.9999432   0.7779119   0.99957812]
  [ 0.42606592 -0.25002331  0.08530296 -0.1822646  -0.24972039]]

 [[-0.99999982 -0.99941862  1.          0.85995275  0.99999774]
  [-0.9999947  -0.99617112  0.99999642 -0.47372907  0.99778104]]

 [[-0.99998683 -0.99998415  0.99890566 -0.87006038  0.01382304]
  [-0.99273354 -0.54457188  0.9352473   0.11324032  0.97421181]]]


In [19]:
# there is a even better idea, dynamic_rnn() function
n_steps=2
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
with tf.variable_scope('B5'):
    outputs, states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
    
X_batch = np.array([
    # t = 0   t = 1
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
])


In [20]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    print(outputs_val)

[[[ 0.26147306  0.92727315  0.46621719  0.81288052  0.69008565]
  [-0.97736037  1.          0.99983335  0.98398542  0.99794364]]

 [[-0.2551924   0.99997282  0.97632003  0.97420961  0.98658937]
  [-0.2595697  -0.27985176  0.43371019  0.2821514  -0.58343893]]

 [[-0.65818655  1.          0.99921155  0.99669862  0.99950302]
  [-0.87030524  0.99998784  0.9984799   0.82325637  0.95229805]]

 [[-0.98587334  0.99694037  0.9997443  -0.99392438 -0.58270109]
  [-0.78614157  0.99458522  0.9759534  -0.32073846  0.61887091]]]


In [3]:
# so far we have used only fixed-size input sequences(all exactly two steps long). 
# What if the input sequences have variable lengths.
import numpy as np
n_steps = 2
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
X_seqs = tf.unstack(tf.transpose(X,perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
seq_length = tf.placeholder(tf.int32,[None])
# there are several cell exists, so made its a name to distinguish
with tf.variable_scope('B2'):
    output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell,X_seqs,dtype=tf.float32,
                                                   sequence_length=seq_length)

outputs = tf.transpose(tf.stack(output_seqs),perm=[1,0,2])

X_batch = np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],  # the [0,0,0] is to maintain the structure, but would not be used because of seq_length
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
])

seq_length_batch = np.array([2,1,2,2])

In [4]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    outputs_val = outputs.eval(feed_dict={X: X_batch,seq_length:seq_length_batch})
    print(outputs_val)

[[[-0.7653302  -0.95297658 -0.60940617  0.14840634  0.82131851]
  [-1.         -0.99998385 -0.99997109 -0.79619229 -0.99885327]]

 [[-0.99991739 -0.99978185 -0.9817003  -0.03475988  0.53253466]
  [ 0.          0.          0.          0.          0.        ]]

 [[-1.         -0.99999917 -0.99929756 -0.21562031  0.02649164]
  [-0.99999815 -0.99436426 -0.99833828 -0.5840103  -0.99366611]]

 [[-0.99998748  0.98282105 -0.57022643 -0.98603582 -0.99996114]
  [-0.99546218 -0.04556185  0.19253059 -0.21432495 -0.61927611]]]


In [52]:
# we will treat each image as a sequence of 28 rows of 28 pixels.
# we will use cells of 150 recurrent neurons, plus a fuly connected layer containing 10 neurons.
# note that the fully connected layer is connected to the states tensor, contain only the final state of RNN.
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.int32,[None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
with tf.variable_scope('B4'):
    outputs, states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)

# Logit is a function that maps probabilities ([0, 1]) to R ((-inf, inf))
logits = tf.layers.dense(states,n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)

loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

init = tf.global_variables_initializer()


In [53]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1,n_steps,n_inputs))
y_test = mnist.test.labels

n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1,n_steps,n_inputs))
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test,y:y_test})
        print(epoch,"Train accuracy",acc_train,"Test accuracy:",acc_test)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 Train accuracy 0.913333 Test accuracy: 0.9342
1 Train accuracy 0.986667 Test accuracy: 0.9482
2 Train accuracy 0.96 Test accuracy: 0.9535
3 Train accuracy 0.98 Test accuracy: 0.9493
4 Train accuracy 0.98 Test accuracy: 0.9642
5 Train accuracy 0.973333 Test accuracy: 0.9706
6 Train accuracy 0.966667 Test accuracy: 0.9662
7 Train accuracy 0.973333 Test accuracy: 0.9666
8 Train accuracy 0.986667 Test accuracy: 0.9723
9 Train accuracy 0.98 Test accuracy: 0.9745


In [24]:
# predict time series
# create RNN that contains 100 recurrent neurons and we will unroll it over 20 time steps
n_steps = 12
n_inputs = 1
n_outputs =1
n_neurons = 100
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None])
#cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,activation=tf.nn.relu)
# this make 100 size output linaer combined to 1.
cell = tf.contrib.rnn.OutputProjectionWrapper(
        tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,activation=tf.nn.relu),output_size = n_outputs)

with tf.variable_scope('B6'):
    outputs, states = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)
    
learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [1]:
import pandas
import math
import numpy as np
n_iterations = 100
batch_size = 50

T = pandas.read_csv('C:/COURSE/time series/Monthly Weather.csv',usecols=[2])
dataset = T.values
dataset = dataset.astype('float32')
# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)
#split to train and test sets
train_size = 240
test_size = 120
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# let X denote the temperature of time t and Y denote the time (t+1)
# convert an array of values into a matrix
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

#reshape into X = t, Y = t+12
look_back = 12
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#reshape input tobe [samples,timesteps,features]
trainX = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1],1))


In [54]:
with tf.Session() as sess:
    sess.run(init)
    for iteration in range(n_iterations):
        X_batch, y_batch = trainX,trainY
        sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
        if iteration % 10 == 0:
            mse = loss.eval(feed_dict={X:X_batch,y:y_batch})
            print(iteration,"\tMSE:",mse)
            
    X_new = testX
    y_pred = sess.run(outputs,feed_dict={X:X_new})
    #print(y_pred)

0 	MSE: 3132.54
10 	MSE: 1615.17
20 	MSE: 796.671
30 	MSE: 737.629
40 	MSE: 714.667
50 	MSE: 685.993
60 	MSE: 658.467
70 	MSE: 637.905
80 	MSE: 617.746
90 	MSE: 603.398


In [ ]:
# Rather than using OutputProjectionWrapper, there is a more effecient way to do so.
cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,activation=tf.nn.relu)
rnn_outputs,states = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)

stacked_rnn_outputs = tf.reshape(rnn_outputs,[-1,n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs,n_outputs)
outputs = tf.reshape(stacked_outputs,[-1,n_steps,n_outputs])

In [ ]:
# Deep RNN
n_neurons = 100
n_layers = 3

layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,activation=tf.nn.relu)
             for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell,X,dtype=tf.float32)

# dropout algorithm
# at every training step, every neuron has a probability p of being temporarily ignored.It typically to be set to 50%.

keep_prob = 0.5

cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
             for layer in range(n_layers)]
cells_drop = [tf.contrib.rnn.DropoutWrapper(cell,input_keep_prob=keep_prob)
             for cell in cells]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells_drop)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell,X,dtype=tf.float32)

# LSTM
lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)